<a href="https://colab.research.google.com/github/ahakobia/Group4_NFLX_MIDTERM/blob/main/Adehs_Pyspark_US_State_Energy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example: 'spark-3.2.2'
spark_version = 'spark-3.2.2'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 261 kB in 3s (76.9 kB/s)
Reading package li

In [2]:
# Install pytest and pytest-sugar to make our output look nice.
!pip install -q pytest pytest-sugar

In [3]:
# Create and navigate to the tdd directory.
from pathlib import Path
if Path.cwd().name != 'tests':
    %mkdir tests
    %cd tests
# Show the current working directory.  
%pwd

mkdir: cannot create directory ‘tests’: File exists
/content/tests


'/content/tests'

In [4]:
# Initialize the __init__.py file. 
# This file will be stored in our pwd (/content/tests)
%%file __init__.py
pass

Overwriting __init__.py


In [5]:
import findspark
findspark.init()

# Import other dependencies. 
from pyspark import SparkFiles
from pyspark.sql import SparkSession

import time
spark = SparkSession.builder.appName("sparkEnergyData").getOrCreate()

# url = "https://raw.githubusercontent.com/ahakobia/Group4_NFLX_MIDTERM/main/Resources/organised_Gen.csv?token=GHSAT0AAAAAABXH6XNLVR2KDPQKXAWUGPNQYZX5EYQ"
# spark.sparkContext.addFile(url)
# energy = spark.read.csv(SparkFiles.get("organised_Gen.csv"), header=True)

spark = SparkSession \
    .builder \
    .appName("energy file read in") \
    .getOrCreate()
energy = spark.read.csv('organised_Gen.csv', header = True)
energy = energy.withColumnRenamed('TYPE OF PRODUCER', 'producer')
energy = energy.withColumnRenamed('ENERGY SOURCE', 'source')
energy = energy.withColumnRenamed('GENERATION (Megawatthours)', 'generated')
energy.write.parquet('parquet_energy',mode='overwrite')
p_energy = spark.read.parquet('parquet_energy')
p_energy.createOrReplaceTempView('p_energy_data')

start_time = time.time()
transformed_df = spark.sql("""
    SELECT
        YEAR as year,
        MONTH as month,
        STATE as state, producer, source, generated
        FROM p_energy_data
        WHERE state != 'US-TOTAL' AND producer == 'Total Electric Power Industry'
        AND source == 'Total'
        """)
transformed_df

  
q_df = spark.sql("""SELECT year, state, (SUM(generated)/1000000) as sums
            FROM p_energy_data 
            WHERE producer = 'Total Electric Power Industry' 
                AND source = 'Total' 
                AND state != 'US-TOTAL'
                AND YEAR != 2022
            Group By YEAR, State
            Order By YEAR DESC, sums DESC
                """)
q_df


print("--- %s seconds ---" % (time.time() - start_time))


--- 0.43701958656311035 seconds ---


In [6]:
q_df.show()

+----+-----+----------+
|year|state|      sums|
+----+-----+----------+
|2021|   TX|483.561772|
|2021|   FL|244.401034|
|2021|   PA| 241.64836|
|2021|   CA|196.727105|
|2021|   IL|181.859368|
|2021|   AL|142.955138|
|2021|   NC|131.262683|
|2021|   GA|126.530744|
|2021|   NY|125.226023|
|2021|   OH|123.335607|
|2021|   MI|116.045383|
|2021|   WA|109.122717|
|2021|   AZ|108.608533|
|2021|   SC| 99.924403|
|2021|   LA| 98.061963|
|2021|   IN| 94.565164|
|2021|   VA| 94.309018|
|2021|   TN| 81.478885|
|2021|   OK| 80.564126|
|2021|   MO| 77.815979|
+----+-----+----------+
only showing top 20 rows



In [16]:
# Create a bank_data.py file and write the function to it. 
# This file will be stored in our pwd (/content/tests).
%%file total_energy.py

# Import findspark() and initialize. 
import findspark
findspark.init()

# Import other dependencies. 
from pyspark import SparkFiles
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sparkEnergyData").getOrCreate()

# Create the import_data function. 
def import_data():
    spark = SparkSession \
    .builder \
    .appName("energy file read in") \
    .getOrCreate()
    energy = spark.read.csv('organised_Gen.csv', header = True)
    energy = energy.withColumnRenamed('TYPE OF PRODUCER', 'producer')
    energy = energy.withColumnRenamed('ENERGY SOURCE', 'source')
    energy = energy.withColumnRenamed('GENERATION (Megawatthours)', 'generated')
    energy.write.parquet('parquet_energy',mode='overwrite')
    p_energy = spark.read.parquet('parquet_energy')
    p_energy.createOrReplaceTempView('p_energy_data')
    return p_energy

def transform_data():
    transformed_df = spark.sql("""
    SELECT
        YEAR as year,
        MONTH as month,
        STATE as state, producer, source, generated
        FROM p_energy_data
        WHERE state == 'US-TOTAL' AND producer == 'Total Electric Power Industry'
        AND source == 'Total'
        """)
    return transformed_df

def query_data():
  
    q_df = spark.sql("""SELECT YEAR, (SUM(generated)/1000000)
            FROM p_energy_data 
            WHERE producer = 'Total Electric Power Industry' 
                AND source = 'Total' 
                AND state = 'US-TOTAL'
                AND YEAR != 2022
            Group By YEAR, State
            Order By YEAR DESC
                """)
    return q_df


Overwriting total_energy.py


In [17]:
# Create a test_bank_data.py file and write the test functions to it. 
# This file will be stored in our pwd (/content/tests).
%%file test_total_states_energy.py

# From the bank_data.py file and import the import_data function. 
from total_states_energy import (import_data, transform_data, query_data)

# Write the tests. 
def test_row_count_before_transform():
  df = import_data()
  assert df.count() == 496774

def test_column_count_before_transform():
  df = import_data()
  assert len(df.columns) == 7

def test_row_count_after_transform():
    df = transform_data()
    assert df.count() != 496774

def test_column_count_after_transform():
    df = transform_data()
    assert len(df.columns) != 7

def test_row_count_query():
    df = query_data()
    assert df.count() == 21

def test_column_count_query():
    df = query_data()
    assert len(df.columns) == 2


Overwriting test_total_states_energy.py


In [18]:
# Run the test_import_data.py file with pytest. 
!python -m pytest test_total_energy.py

Test session starts (platform: linux, Python 3.7.14, pytest 3.6.4, pytest-sugar 0.9.5)
rootdir: /content/tests, inifile:
plugins: typeguard-2.7.1, sugar-0.9.5

 test_total_energy.py ✓✓✓✓✓✓                                     100% ██████████

Results (30.29s):
       6 passed
